In [ ]:
import os
import pickle

import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
import seaborn as sns

# Load Data

In [ ]:
directory = "INSERT DIRECTORY" # Directory containing zcr and ma results files
ae_dir = "INSERT DIRECTORY" # Directory containing ae results file

with open(os.path.join(ae_dir, 'INSERT FILENAME'), 'rb') as f:
    ae_results_df = pickle.load(f)
with open(os.path.join(directory, 'INSERT FILENAME'), 'rb') as f:
    sw_results_df = pickle.load(f)
with open(os.path.join(directory, 'INSERT FILENAME'), 'rb') as f:
    zcr_results_df = pickle.load(f)

# Preprocess Data

In [ ]:
ae_results_df_mean = ae_results_df.reset_index()
ae_results_df_mean = ae_results_df_mean.groupby(['SNR', 'Thresh']).agg('mean')
ae_results_df_mean.head()

In [ ]:
sw_results_df_mean = sw_results_df.reset_index()
sw_results_df_mean = sw_results_df_mean.groupby(['SNR', 'Thresh']).agg('mean')
sw_results_df_mean.head()

In [ ]:
zcr_results_df_mean = zcr_results_df.reset_index()
zcr_results_df_mean = zcr_results_df_mean.groupby(['SNR', 'Thresh']).agg('mean')
zcr_results_df_mean.head()

In [ ]:
ae_results_df_std = ae_results_df.reset_index()
ae_results_df_std = ae_results_df_std.groupby(['SNR', 'Thresh']).agg('std')
ae_results_df_std.head()

In [ ]:
sw_results_df_std = sw_results_df.reset_index()
sw_results_df_std = sw_results_df_std.groupby(['SNR', 'Thresh']).agg('std')
sw_results_df_std.head()

In [ ]:
zcr_results_df_std = zcr_results_df.reset_index()
zcr_results_df_std = zcr_results_df_std.groupby(['SNR', 'Thresh']).agg('std')
zcr_results_df_std.head()

In [ ]:
# Calculate AUC for each result
from sklearn.metrics import auc

def auc_calc(group):
    group = group.sort_values(by='FPR')
    
    fprs = group.FPR.values
    tprs = group.TPR.values
    val = auc(fprs, tprs)
    
    return val
    

zcr_auc_df = zcr_results_df.groupby(['SNR', 'trace_idx']).apply(auc_calc).rename('AUC')
zcr_auc_df = zcr_auc_df.to_frame()
ae_auc_df = ae_results_df.groupby(['SNR', 'trace_idx']).apply(auc_calc).rename('AUC')
ae_auc_df = ae_auc_df.to_frame()
sw_auc_df = sw_results_df.groupby(['SNR', 'trace_idx']).apply(auc_calc).rename('AUC')
sw_auc_df = sw_auc_df.to_frame()
sw_auc_df

# Plots

In [ ]:
# Plot a comparison of the AUC performances for each technique
fig, axs = plt.subplots(1, 3, figsize=(6, 2.75), dpi=600, sharey=True)
sns.barplot(data=zcr_auc_df.reset_index(), x='SNR', y='AUC', palette='tab10', ax=axs[0])
sns.barplot(data=ae_auc_df.reset_index(), x='SNR', y='AUC', palette='tab10', ax=axs[1])
sns.barplot(data=sw_auc_df.reset_index(), x='SNR', y='AUC', palette='tab10', ax=axs[2])

axs[0].set_ylabel("AUC", weight='bold')
axs[1].set_ylabel("")
axs[2].set_ylabel("")

labels = ['(a)', '(b)', '(c)']

for i, ax in enumerate(axs):
    ax.set_xlabel("SNR", weight='bold')
    ax.tick_params(width=1.5)
    for axis in ['top', 'right', 'bottom', 'left']:
        ax.spines[axis].set_linewidth(1.5)
    for label in ax.get_xticklabels():
        label.set_rotation(90)
    ax.annotate(labels[i], xy=(0.1, 0), xytext=(-0.3, 1.15), weight='bold')
    
# axs[0].set(title='zcr')
# axs[1].set(title='ae')
# axs[2].set(title='sw')

fig.tight_layout()
# fig.savefig('Results.png')

In [ ]:
sw_auc_df.groupby('SNR').agg('mean')